# Machine Learning Challenge #3

## Predict Ad Clicks 

In [1]:
# Import Libraries

import pandas as pd
import numpy as np

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Documentation
import handcalcs.render

# Plot
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm # color map
import seaborn as sns

from sympy import Sum, symbols, Indexed, lambdify, diff
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d.axes3d import Axes3D
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

ModuleNotFoundError: No module named 'pandas'

In [3]:
# Data
data = 'Data/'

train_df = pd.read_csv(data+'train.csv')
test_df = pd.read_csv(data+'test.csv')
sample_submission_df = pd.read_csv(data+'sample_submission.csv')

In [4]:
print('Shape of train data: ', train_df.shape)
print('Shape of test data: ', test_df.shape)
print('Shape of sample submission data: ', sample_submission_df.shape)

Shape of train data:  (12137810, 10)
Shape of test data:  (3706907, 9)
Shape of sample submission data:  (3706907, 2)


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12137810 entries, 0 to 12137809
Data columns (total 10 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ID           object 
 1   datetime     object 
 2   siteid       float64
 3   offerid      int64  
 4   category     int64  
 5   merchant     int64  
 6   countrycode  object 
 7   browserid    object 
 8   devid        object 
 9   click        int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 926.0+ MB


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3706907 entries, 0 to 3706906
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ID           object 
 1   datetime     object 
 2   siteid       float64
 3   offerid      int64  
 4   category     int64  
 5   merchant     int64  
 6   countrycode  object 
 7   browserid    object 
 8   devid        object 
dtypes: float64(1), int64(3), object(5)
memory usage: 254.5+ MB


In [9]:
sample_submission_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3706907 entries, 0 to 3706906
Data columns (total 2 columns):
 #   Column  Dtype  
---  ------  -----  
 0   ID      object 
 1   click   float64
dtypes: float64(1), object(1)
memory usage: 56.6+ MB


In [10]:
train_df.describe()

,siteid,offerid,category,merchant,click
count,1.092530e+07,1.213781e+07,1.213781e+07,1.213781e+07,1.213781e+07
mean,5.023971e+06,4.966571e+05,4.215284e+04,4.901602e+07,3.602083e-02
std,2.904255e+06,2.897379e+05,2.950048e+04,2.916656e+07,1.863420e-01
min,7.100000e+01,0.000000e+00,9.700000e+01,2.604430e+05,0.000000e+00
25%,2.506733e+06,2.475320e+05,1.496000e+04,2.639177e+07,0.000000e+00
50%,4.984805e+06,4.957550e+05,4.033900e+04,4.861553e+07,0.000000e+00
75%,7.582930e+06,7.472350e+05,6.678600e+04,7.436361e+07,0.000000e+00
max,9.999996e+06,9.999980e+05,9.991400e+04,9.977043e+07,1.000000e+00


In [12]:
test_df.describe()

,siteid,offerid,category,merchant
count,3.336131e+06,3.706907e+06,3.706907e+06,3.706907e+06
mean,5.046957e+06,4.855628e+05,4.090971e+04,4.868378e+07
std,2.848010e+06,2.945176e+05,2.833568e+04,2.754481e+07
min,1.110000e+02,0.000000e+00,9.700000e+01,2.604430e+05
25%,2.627839e+06,2.283750e+05,1.496000e+04,2.772386e+07
50%,5.053327e+06,4.711990e+05,4.164000e+04,4.377045e+07
75%,7.512147e+06,7.443430e+05,5.554100e+04,7.208974e+07
max,9.999830e+06,9.999980e+05,9.991400e+04,9.977043e+07


In [13]:
sample_submission_df.describe()

,click
count,3706907.0
mean,0.5
std,0.0
min,0.5
25%,0.5
50%,0.5
75%,0.5
max,0.5


In [14]:
train_df.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,click
0,IDsrk7SoW,2017-01-14 09:42:09,4709696.0,887235,17714,20301556,e,Firefox,NaN,0
1,IDmMSxHur,2017-01-18 17:50:53,5189467.0,178235,21407,9434818,b,Mozilla Firefox,Desktop,0
2,IDVLNN0Ut,2017-01-11 12:46:49,98480.0,518539,25085,2050923,a,Edge,NaN,0
3,ID32T6wwQ,2017-01-17 10:18:43,8896401.0,390352,40339,72089744,c,Firefox,Mobile,0
4,IDqUShzMg,2017-01-14 16:02:33,5635120.0,472937,12052,39507200,d,Mozilla Firefox,Desktop,0


In [15]:
test_df.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid
0,IDFDJVI,2017-01-22 09:55:48,755610.0,808980,17714,26391770,b,Mozilla Firefox,Desktop
1,IDNWkTQ,2017-01-22 03:54:39,3714899.0,280355,12052,39507200,b,Edge,Tablet
2,ID9pRmM,2017-01-21 10:25:50,4378333.0,930819,30580,46148550,e,Mozilla Firefox,NaN
3,IDHaQaj,2017-01-22 14:45:53,1754730.0,612234,11837,8837581,b,Edge,Tablet
4,IDT2CrF,2017-01-22 09:34:07,5299909.0,524289,45620,31388981,b,Mozilla,NaN


In [16]:
sample_submission_df.head()

,ID,click
0,IDE4beP,0.5
1,IDfo26Y,0.5
2,IDYZM6I,0.5
3,ID8CVw1,0.5
4,IDPltMK,0.5


In [17]:
# Checking NaN values

train_df.isnull().sum()

ID                   0
datetime             0
siteid         1212515
offerid              0
category             0
merchant             0
countrycode          0
browserid       608327
devid          1820299
click                0
dtype: int64

In [18]:
test_df.isnull().sum()

ID                  0
datetime            0
siteid         370776
offerid             0
category            0
merchant            0
countrycode         0
browserid      221906
devid          704619
dtype: int64